# Plots

In [ ]:
import os

import pandas as pd
import numpy as np
from itertools import product
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import plotly.graph_objects as go

from utils import SankeyPlot

dpi = 600
sns.reset_defaults()

In [ ]:
# Import dataset
df = pd.read_excel("data/dataset_fda_devices.xlsx")

In [ ]:
# Dates to datetime format
date_columns = [col for col in df.columns if "Date" in col]
df[date_columns] = df[date_columns].apply(pd.to_datetime)
df["Year"] = df["Date of Final Decision"].dt.year

In [ ]:
df = df.replace({"summary not available": np.NAN})

In [ ]:
df[["Clinical Performance Study"]].value_counts()

In [ ]:
df[["Submission Number", "Device", "Company", "Recall", "Date of Final Decision", "FDA Recall Posting Date", "Clinical Performance Study"]].to_csv("fda_devices_dataset.csv", index=False)

In [ ]:
os.makedirs("plots", exist_ok=True)

## Figure 1
Total number of available FDA-approved AI-enabled medical devices by country (above) and percentage of recalled devices after FDA-approval by country (below).

In [ ]:
# Create dataframe with FDA-approved devices count per country
df["count"] = np.ones(len(df))
df_counts = df.groupby("Country Applicant").sum(numeric_only=True)[["count"]]
df_alpha_codes = pd.read_csv("data/country_alpha_codes.csv")
df_alpha_codes = df_alpha_codes.rename({"Country": "Country Applicant", "Alpha-3 code": "alpha_code"}, axis=1)
df_alpha_codes["alpha_code"] = df_alpha_codes["alpha_code"].apply(lambda x: x.replace('"', ''))
df_alpha_codes["alpha_code"] = df_alpha_codes["alpha_code"].apply(lambda x: x.replace(" ", ""))
df_counts = df_counts.merge(df_alpha_codes[["Country Applicant", "alpha_code"]], on="Country Applicant", how="left")
df_counts.drop(df_counts[df_counts["Country Applicant"]=="not accessible"].index, inplace=True)
df_counts["alpha_code"] = df_counts["alpha_code"].apply(lambda x: x.replace(" ", ""))
df_counts.head()

In [ ]:
# Create dataframe to count recalled devices per country
df_counts_recall = df[df["Recall"]=="yes"].groupby("Country Applicant").sum(numeric_only=True)[["count"]]
df_counts_recall = df_counts_recall.merge(df_alpha_codes[["Country Applicant", "alpha_code"]], on="Country Applicant", how="left")
df_counts_recall.drop(df_counts_recall[df_counts_recall["Country Applicant"]=="not accessible"].index, inplace=True)
df_counts_recall["alpha_code"] = df_counts_recall["alpha_code"].apply(lambda x: x.replace(" ", ""))
df_counts_recall.head()

In [ ]:
# Count by year
df_counts_year = df.groupby(["Country Applicant", "Year"]).sum("count")
df_counts_year.reset_index(inplace=True)
df_counts_year = df_counts_year.sort_values(by=["Country Applicant", "Year"])
df_counts_year["cum_count"] = df_counts_year.groupby(["Country Applicant"]).cumsum()["count"]
df_counts_year = df_counts_year.merge(df_alpha_codes[["Country Applicant", "alpha_code"]], on="Country Applicant", how="left")
df_counts_year.drop(df_counts_year[df_counts_year["Country Applicant"]=="not accessible"].index, inplace=True)
df_counts_year = df_counts_year.sort_values("Year")
df_counts_year.head()

In [ ]:
# Cumulative count by year
country_year = pd.DataFrame(list(product(df_counts_year["Country Applicant"].unique(), df["Year"].unique())), columns=["Country Applicant", "Year"])
country_year = country_year.merge(df_alpha_codes[["Country Applicant", "alpha_code"]], on="Country Applicant", how="left")
country_year = country_year.merge(df_counts_year[["Country Applicant", "Year", "cum_count"]], on=("Country Applicant", "Year"), how="left")
min_year = df_counts_year.groupby("Country Applicant").min()["Year"]
for country, year in min_year.items():
    country_year.drop(country_year[(country_year["Country Applicant"]==country) & (country_year["Year"]<year)].index, inplace=True)
for country in country_year["Country Applicant"].unique():
    country_year[country_year["Country Applicant"] == country] = country_year[country_year["Country Applicant"] == country].bfill()

country_year.sort_values(["Year"], inplace=True)
country_year.sort_values(["Country Applicant", "Year"]).head()

In [ ]:
df.drop("count", axis=1, inplace=True)
df = df.merge(df_counts[["Country Applicant", "alpha_code"]], on="Country Applicant", how="left")
country_year.rename({"cum_count": "Approved devices"}, axis=1, inplace=True)

In [ ]:
df_counts_recall_perc = df_counts_recall[["Country Applicant", "count"]].merge(df_counts[["count", "Country Applicant", "alpha_code"]], on="Country Applicant", suffixes=("_recalled", "_tot"), how="right").fillna(0)
df_counts_recall_perc["perc"] =  df_counts_recall_perc["count_recalled"].div(df_counts_recall_perc["count_tot"]) * 100

colors = ["Viridis", "YlOrRd_r"]
names = ["all", "recalled"]
legend_titles = ["Approved devices", "Recalled devices"]
for i, (df_plot, to_plot) in enumerate(zip([df_counts, df_counts_recall_perc], ["count", "perc"])):
    fig = go.Figure(data=go.Choropleth(
        locations = df_plot['alpha_code'],
        z = df_plot[to_plot],
        text = df['Country Applicant'],
        colorscale = colors[i],
        autocolorscale=False,
        reversescale=True,
        marker_line_color='darkgray',
        marker_line_width=0.5,
        colorbar_title = legend_titles[i],
        colorbar_len = 0.6
    ))
    
    fig.update_layout(
        margin={"r":0,"t":0,"l":0,"b":0},
        # title_text='2014 Global GDP',
        geo=dict(
            showframe=False,
            showcoastlines=True,
            projection_type='natural earth'
        )
    )
    fig.write_image(f"plots/Figure1_{i}.pdf", scale=2, format="pdf")
    fig.show()

## Figure 2
Number of AI-enabled medical devices by specialty, along with details on the design of their clinical performance studies.

In [ ]:
df["Clinical Performance Study"].value_counts(dropna=False)

In [ ]:
df_x = df.copy()
df_x = df_x[df_x["Clinical Performance Study"]=="yes"]  # select only devices for which a CP study was performed
df_x["order"] = df_x["specialty"].apply(lambda x: {'Cardiovascular': 1, 'Hematology': 3, 'Neurology': 2, 'Radiology': 0, 'Other': 4}[x])
df_x = df_x.sort_values("order")
df_x.loc[df_x["CP Study-type"]=="No", "CP Study-type"] = "Not Specified"
df_x["CP Study-type"] = df_x["CP Study-type"].replace({"Missing": "Missing info"})
df_x["CP Study-type"] = df_x["CP Study-type"].astype("category")
df_x["CP Study-type"] = df_x["CP Study-type"].cat.remove_unused_categories() 

In [ ]:
sns.reset_defaults()
sns.set_style("white", {'figure.facecolor': 'white'})
fig, ax = plt.subplots(1, figsize=(8, 5))
sns.set(font_scale=1.3)
sns.histplot(df_x, y="specialty", hue="CP Study-type", multiple="dodge", fill=True, shrink=0.6, ax=ax)
ax.set_xlabel("Number of devices", fontsize=14)
ax.set_ylabel("")
ax.tick_params(axis='y', which='major', labelsize=15)
legend = ax.get_legend()
frame = legend.get_frame()
frame.set_facecolor("white")
legend.set_title("Clinical performance study")
plt.savefig("plots/Figure2.pdf", dpi=dpi, bbox_inches='tight', format="pdf")
sns.set(font_scale=1)
plt.show()

## Figure 3
Performance distribution and relationships among AUC, sensitivity, and specificity metrics for AI-enabled medical devices in clinical performance studies.

In [ ]:
df = df.rename({"AUC_cp": "AUC", "Sens_cp": "Sensitivity", "Spec_cp": "Specificity"}, axis=1)
df["Recall"] = df["Recall"].replace({"no": "Available device", "yes": "Recalled device"})

In [ ]:
metrics = ["AUC", "Sensitivity", "Specificity"]
for metric in metrics:
    df[metric] = pd.to_numeric(df[metric], errors="coerce")

In [ ]:
sns.reset_defaults()
a = sns.pairplot(df[metrics+["Recall"]], hue="Recall", plot_kws={"alpha": 0.5},
                 diag_kind="hist", diag_kws={"binwidth": 0.04, "stat": "density", "linewidth": 0.5})
for ax in a.axes[2, :]:
    ax.set_xlabel(ax.get_xlabel(), fontweight="bold")
for ax in a.axes[:, 0]:
    ax.set_ylabel(ax.get_ylabel(), fontweight="bold")

a.axes[0,1].clear()
a.axes[0,1].set_axis_off()
a.axes[0,2].clear()
a.axes[0,2].set_axis_off()
a.axes[1,2].clear()
a.axes[1,2].set_axis_off()

for ax_ in a.axes:
    for i, ax in enumerate(ax_):
        ax.set_xlim(0.15)
        ax.set_ylim(0.2)

handles = a._legend_data.values()
labels = a._legend_data.keys()
a._legend.remove()
a.axes[0,2].legend(handles=handles, labels=labels, loc='center right', ncol=1)
plt.savefig("plots/Figure3.pdf", dpi=dpi, format="pdf")
plt.show()

## Figure 4
Time lag between approval and recall of the devices.

In [ ]:
df_recall = df.loc[df["Recall"]=="Recalled device", ["Date of Final Decision", "FDA Recall Posting Date", "Device", "Physical State", "Submission Number"]].reset_index(drop=True)

In [ ]:
# Legend handles
line_software, line_other = None, None
dot_circle, dot_x = None, None
fig, ax = plt.subplots(1, figsize=(10, 15))
for i, row in df_recall.iterrows():
    i_color = 1 if row["Physical State"] == "Software" else 0
    line = ax.plot(row.iloc[0:2].values, [i, i], '-', linewidth=4, color=list(plt.cm.tab10.colors)[i_color],
                   label="Software" if i_color == 1 and line_software is None else
                         "Software + Device" if i_color == 0 and line_other is None else None)
    if i_color == 1 and line_software is None:
        line_software = line  # Store reference for legend
    elif i_color == 0 and line_other is None:
        line_other = line
    dot_o = plt.plot(row.iloc[0], [i], marker='o', markersize=6, color=list(plt.cm.tab10.colors)[2], markeredgewidth=3,
                     label="Approval Date" if dot_circle is None else None, linestyle='None')
    dot_x = plt.plot(row.iloc[1], [i], marker='x', markersize=7, color=list(plt.cm.tab10.colors)[3], markeredgewidth=3,
                     label="Recall Date" if dot_x is None else None, linestyle='None')
    if dot_circle is None:
        dot_circle = dot_o
    if dot_x is None:
        dot_x = dot_x
    ax.hlines(y=i, xmin=df_recall["Date of Final Decision"].min(), xmax=row.iloc[0], alpha=0.1, color="black", ls=":")

plt.yticks(np.arange(len(df_recall)), df_recall["Submission Number"])
plt.ylabel("")
plt.xlabel("Year")
handles, labels = plt.gca().get_legend_handles_labels()
order = [3,0,1,2]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order], fontsize=14)
plt.savefig("plots/Figure4.pdf", dpi=dpi, bbox_inches='tight', format="pdf")
plt.show()

## Appendix 1
Cumulative histogram of available and recalled AI-enabled medical devices that received FDA approval over the period 1995-2024.

In [ ]:
date_0 = df[df["Recall"]=="Available device"]["Date of Final Decision"].sort_values().values
date_1 = df[df["Recall"]=="Recalled device"]["Date of Final Decision"].sort_values().values
count_0 = np.arange(1, len(date_0) +1 )
count_1 = np.arange(1, len(date_1) +1 )
fig, ax = plt.subplots(1)
plt.plot(date_0, count_0, linewidth=2.5, label="0", zorder=1)
plt.plot(date_1, count_1, linewidth=2.5, label="1", zorder=0)
plt.xlabel("Year of approval", fontsize=12)
plt.ylabel("Number of devices", fontsize=12)
plt.grid(alpha=0.5, axis="y")
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.legend(["Available devices", "Recalled devices"], title="")
plt.savefig("plots/Appendix1.pdf", dpi=600, format="pdf")
plt.show()

## Appendix 2
Total number of available FDA-approved AI-enabled medical devices by country (panel above). Total number of recalled devices after FDA-approval by country (panel below).

In [ ]:
colors = ["Viridis", "YlOrRd_r"]
names = ["all", "recalled"]
legend_titles = ["Approved devices", "Recalled devices"]
for i, df_plot in enumerate([df_counts, df_counts_recall]):
    fig = go.Figure(data=go.Choropleth(
        locations = df_plot['alpha_code'],
        z = df_plot['count'],
        text = df['Country Applicant'],
        colorscale = colors[i],
        autocolorscale=False,
        reversescale=True,
        marker_line_color='darkgray',
        marker_line_width=0.5,
        colorbar_title = legend_titles[i],
        colorbar_len = 0.6
    ))
    
    fig.update_layout(
        margin={"r":0,"t":0,"l":0,"b":0},
        geo=dict(
            showframe=False,
            showcoastlines=True,
            projection_type='natural earth'
        )
    )
    fig.write_image(f"plots/Appendix2_{i}.pdf", scale=2)
    fig.show()

## Appendix 3
Distribution of AI-enabled medical devices availability and physical state across the available specialties.

In [ ]:
df_sankey = df.copy()
df_sankey["approved"] = "FDA approved"
df_sankey["Recall"] = df_sankey["Recall"].map({"Recalled device": "Recalled", "Available device": "Still available"})
sankey_recalled = SankeyPlot(df_sankey, target_list=["approved", "Recall", "Physical State", "specialty"])
fig = sankey_recalled.plot_sankey("plots/Appendix3.pdf", show_numbers=True, title=None)
fig

## Appendix 4
Number of clinical performance studies by the physical state of the AI-enabled medical device in separate for recalled and available AI-enabled medical devices.

In [ ]:
cp_features = ["Clinical Performance Study", "CP Study-type", "CP N samples",
               "CP N clinical sites ", "CP results sex available", "CP results age available",
               "CP detailed result values available", "AUC", "Sensitivity", "Specificity"]
df_cp = df[cp_features + ["Geographic area", "Recall", "specialty", "Physical State"]].replace(
    {"summary not available": np.nan})
df_cp = df_cp.fillna("Missing info")

In [ ]:
df_cp["Clinical Performance Study"] = df_cp["Clinical Performance Study"].apply(str.capitalize)

df_cp["order"] = df_cp["Clinical Performance Study"].apply(lambda x: {'Yes': 0, 'No': 1, 'Missing info': 2}[x])
df_cp = df_cp.sort_values("order")

In [ ]:
sns.histplot(df_cp, hue="Clinical Performance Study", x="Physical State", multiple="dodge", shrink=.7, stat="count", common_norm=True)
plt.savefig("plots/Appendix4_all.pdf", dpi=dpi, bbox_inches='tight', format="pdf")
plt.show()

In [ ]:
sns.reset_defaults()
df_cp["Recall"] = df_cp["Recall"].replace({"no": "Available device", "yes": "Recalled device"})
fig, ax = plt.subplots(1, 2, figsize=(13, 5), sharey=False)
# Available
sns.histplot(df_cp[df_cp["Recall"]=="Recalled device"], hue="Clinical Performance Study", x="Physical State", multiple="dodge", shrink=.7, stat="count", common_norm=True, ax=ax[0])
# ax[0].legend(loc="upper right", title="Clinical Performance Study")
ax[0].set_title("Recalled", fontweight="bold")
ax[0].set_ylabel("Number of devices")

sns.histplot(df_cp[df_cp["Recall"]=="Available device"], hue="Clinical Performance Study", x="Physical State", multiple="dodge", shrink=.7, stat="count", common_norm=True, ax=ax[1])
# ax[1].legend(["Yes", "Missing info", "No"], loc="upper left", title="Clinical Performance Study")
# ax[1].get_legend().remove()
ax[1].set_title("Available", fontweight="bold")
ax[1].set_ylabel("Number of devices")

plt.subplots_adjust(wspace=0.2)
plt.savefig("plots/Appendix4_recalled_available.pdf", dpi=dpi, bbox_inches='tight', format="pdf")
plt.show()

## Appendix 5
Number of FDA-approved AI-enabled medical devices by specialty, together with details of clinical performance (CP) study type. Recalled devices are reported on the left-hand panel, while still-available devices are on the right-hand panel.

In [ ]:
sns.reset_defaults()
fig, ax = plt.subplots(1,2, figsize=(20, 8))
sns.set(font_scale=1.3)
sns.histplot(df_x[df_x["Recall"]=="yes"], y="specialty", hue="CP Study-type", multiple="dodge", fill=True, shrink=0.6, ax=ax[0], legend=False)
sns.histplot(df_x[df_x["Recall"]=="no"], y="specialty", hue="CP Study-type", multiple="dodge", fill=True, shrink=0.6, ax=ax[1])
ax[0].set_title("Recalled", fontsize=16, fontweight="bold")
ax[1].set_title("Available", fontsize=16, fontweight="bold")
ax[0].set_xlabel("Number of devices", fontsize=14)
ax[1].set_xlabel("Number of  devices", fontsize=14)
ax[0].set_ylabel("")
ax[1].set_ylabel("")
ax[0].tick_params(axis='y', which='major', labelsize=15)
ax[0].xaxis.set_major_locator(MaxNLocator(integer=True))
ax[1].axes.yaxis.set_ticklabels([])
sns.move_legend(ax[1], loc="lower right", facecolor="white")
ax[1].get_legend().set_title("Clinical performance study")
plt.subplots_adjust(wspace=0.05)
plt.savefig("plots/Appendix5.pdf", dpi=dpi, bbox_inches='tight', format="pdf")
plt.show()